In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 20:28].values
classes = np.reshape(classes, (2000, 700, 8))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 8)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(8, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accu = []
precisions = []
recalls = []
f1 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 8))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 8))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0818 11:08:02.610097 11104 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0818 11:08:02.616066 11104 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0818 11:08:02.617049 11104 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    1    2 ... 1997 1998 1999] TEST: [  11   21   35   49   53   56   57   79   82   86   99  107  111  114
  115  118  119  133  138  139  143  151  158  160  161  163  165  170
  171  174  186  188  189  191  202  208  211  215  218  220  222  224
  225  227  229  230  232  253  254  258  268  270  278  281  284  287
  292  294  306  311  319  323  325  331  333  337  340  343  346  374
  379  384  400  403  406  407  412  413  416  434  435  437  442  446
  448  452  462  463  467  470  475  479  482  485  488  496  502  506
  509  514  518  519  520  526  531  542  547  562  563  567  570  573
  577  593  595  599  606  613  615  622  623  624  628  630  633  635
  643  645  654  657  658  668  674  676  682  684  686  691  696  701
  705  709  711  712  730  735  739  743  744  748  749  751  756  758
  760  772  778  780  781  786  789  790  794  802  804  809  813  815
  818  821  822  824  825  826  846  848  853  855  857  859  867  872
  883  885  886  891  892  8

W0818 11:08:03.419931 11104 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.6211 - acc: 0.1400
Epoch 2/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.5564 - acc: 0.1610
Epoch 3/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.5296 - acc: 0.1715
Epoch 4/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.5190 - acc: 0.1762
Epoch 5/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.5129 - acc: 0.1796
Epoch 6/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.5093 - acc: 0.1815
Epoch 7/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.5027 - acc: 0.1845
Epoch 8/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.4968 - acc: 0.1870
Epoch 9/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.4930 - acc: 0.1884
Epoch 10/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.4918 - acc: 0.1893

1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3152 - acc: 0.2498
Epoch 83/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3152 - acc: 0.2498
Epoch 84/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3133 - acc: 0.2507
Epoch 85/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3109 - acc: 0.2514
Epoch 86/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3101 - acc: 0.2517
Epoch 87/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3098 - acc: 0.2516
Epoch 88/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3107 - acc: 0.2514
Epoch 89/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3145 - acc: 0.2501
Epoch 90/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3081 - acc: 0.2525
Epoch 91/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3059 - acc: 0.2532
Epo

...

c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1177
           1       0.61      0.65      0.63     21638
           2       0.34      0.24      0.28      4094
           3       0.69      0.76      0.73     33410
           4       0.36      0.21      0.27       690
           5       0.42      0.54      0.48     20459
           6       0.36      0.06      0.10      9029
           7       0.38      0.38      0.38     11719

    accuracy                           0.56    102216
   macro avg       0.40      0.36      0.36    102216
weighted avg       0.53      0.56      0.53    102216

Acurácia
0.3559423813073241
Precisao
0.5338225271644209
Recall
0.5560284104249824
F1
0.5329092123930853
[[    0   208    24   208     5   562    34   136]
 [    0 14138   250  2661    36  3587   112   854]
 [    0   468   981   976    10  1018    58   583]
 [    0  2372   481 25488   125  3071    88  1785]
 [    0    70    11   345   146    78    

1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3806 - acc: 0.2336
Epoch 51/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3791 - acc: 0.2339
Epoch 52/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3761 - acc: 0.2348
Epoch 53/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3731 - acc: 0.2359
Epoch 54/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3713 - acc: 0.2363
Epoch 55/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3694 - acc: 0.2370
Epoch 56/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3658 - acc: 0.2379
Epoch 57/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3639 - acc: 0.2387
Epoch 58/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3625 - acc: 0.2392
Epoch 59/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3594 - acc: 0.2401
Epo

Epoch 1/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.6215 - acc: 0.1395
Epoch 2/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.5625 - acc: 0.1600
Epoch 3/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.5343 - acc: 0.1702
Epoch 4/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.5207 - acc: 0.1760
Epoch 5/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.5150 - acc: 0.1789
Epoch 6/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.5070 - acc: 0.1828
Epoch 7/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.5012 - acc: 0.1857
Epoch 8/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.5000 - acc: 0.1864
Epoch 9/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.4941 - acc: 0.1891
Epoch 10/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.4926 - acc: 0.1892

1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3174 - acc: 0.2496
Epoch 83/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3150 - acc: 0.2505
Epoch 84/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3134 - acc: 0.2506
Epoch 85/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3129 - acc: 0.2510
Epoch 86/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3124 - acc: 0.2513
Epoch 87/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3145 - acc: 0.2506
Epoch 88/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3092 - acc: 0.2522
Epoch 89/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3078 - acc: 0.2528
Epoch 90/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3066 - acc: 0.2530
Epoch 91/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3059 - acc: 0.2534
Epo

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_8 (Bidirection (None, 700, 200)          97600     
_________________________________________________________________
dropout_8 (Dropout)          (None, 700, 200)          0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 700, 200)          241600    
_________________________________________________________________
dropout_9 (Dropout)          (None, 700, 200)          0         
_________________________________________________________________
dense_4 (Dense)              (None, 700, 8)            1608      
Total params: 340,808
Trainable params: 340,808
Non-trainable params: 0
_________________________________________________________________


In [10]:
print('Acurácias total')
print(accu)
a = np.array(accu)
print(a.mean())
print('Precision total')
print(precisions)
p = np.array(precisions)
print(p.mean())
print('Recalls total')
print(recalls)
r = np.array(recalls)
print(r.mean())
print('F1 total')
print(f1)
f = np.array(f1)
print(f.mean())

Acurácias total
[0.3371989476133228, 0.3529153921064021, 0.3559423813073241, 0.3487471656498319, 0.36109926613388743]
0.35118063056215365
Precision total
[0.5182006635183862, 0.5362492531929546, 0.5338225271644209, 0.5298425077236827, 0.5454382647049127]
0.5327106432608714
Recalls total
[0.536523037460998, 0.5545699560262897, 0.5560284104249824, 0.5467952011120849, 0.5528282673020021]
0.5493489744652714
F1 total
[0.5158061111484329, 0.5336454678925882, 0.5329092123930853, 0.5248162391379048, 0.531516504154013]
0.5277387069452047
